# BERT feature extraction for each given dataset

## Librares and utilities

In [8]:
!pip install tape_proteins
#!pip install awscli --ignore-installed six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import torch
from tape import ProteinBertModel, TAPETokenizer
from tape import UniRepModel
import numpy as np
model = ProteinBertModel.from_pretrained('bert-base')
tokenizer = TAPETokenizer(vocab='iupac')

In [10]:
def clip_long_seq(seq, max_len):
  if(len(seq) > max_len):
    seq = seq[0:max_len]
  return seq
def pad_sequence(seq,max_len):
  while(len(seq) < max_len):
    seq = seq + '0'
  return seq

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Feature vector for Dataset 1

In [12]:
entries = []
with  open('/content/drive/MyDrive/Database.txt') as fp:
    contents = fp.read()
    for entry in contents.split('-'):
      entry = entry.replace('\n','')
      entries.append(entry)
entries.pop()
'''
final_entries = [" ".join(entry) for entry in entries]
spaced_amino_p = [char+' ' for entry in entries for char in entry]
amino_string_p = "".join(spaced_amino_p)
'''

'\nfinal_entries = [" ".join(entry) for entry in entries]\nspaced_amino_p = [char+\' \' for entry in entries for char in entry]\namino_string_p = "".join(spaced_amino_p)\n'

In [ ]:
#test_entries = entries[1:10]
ctr = 0
file_write = open("/content/drive/MyDrive/output_primary.txt","w")
for entry in entries:
  entry = clip_long_seq(entry, 1024)
  token_ids = torch.tensor([tokenizer.encode(entry)])
  output = model(token_ids)
  sq = torch.squeeze(output[0],dim=0)
  avg_output = torch.mean(sq,0)
  avg_output_corr = torch.clip(avg_output, min=-1,max=1)
  #print(torch.mean(avg_output_corr))
  ctr+=1
  file_write.write(str(avg_output_corr) + "\n")
  #pooled_output = output[1]
  #file_write.write(str(avg_output_corr) + "\n")
  

## Feature vector for secondary structure predictions

In [15]:
prediction_file = '/content/drive/MyDrive/prediction.txt'
file1 = open(prediction_file, 'r')
Lines = file1.readlines()
#final_lines = [" ".join(line) for line in Lines]
#spaced_amino_s = [char+' ' for line in Lines for char in line]
#amino_string_s = "".join(spaced_amino_s)

In [ ]:
sequences = []
ctr = 0
test_lines = Lines[0:10]
file_write = open("output_secondary.txt","w")
for line in Lines:
  line_str = line.rstrip("\n")
  line_str = clip_long_seq(line_str, 1024)
  token_ids = torch.tensor([tokenizer.encode(line_str)])
  output = model(token_ids)
  sq = torch.squeeze(output[0],dim=0)
  avg_output = torch.mean(sq,0)
  avg_output_corr = torch.clip(avg_output, min=-1,max=1)
  #print(torch.mean(avg_output_corr))
  ctr += 1
  file_write.write(str(avg_output_corr) + "\n")
  #pooled_output = output[1]
  #file_write.write(str(avg_output_corr) + "\n")

In [ ]:
print(len(final_lines))
print(len(final_lines[1]))
print(len(final_lines[1].rstrip("\n")))

85450
2049
2048
